<a href="https://colab.research.google.com/github/Dipak22/DeepLearning/blob/master/sentence_comparision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets numpy evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [4]:
raw_datasets = load_dataset('glue','mrpc')
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def tokenize_function(example):
  return tokenizer(example['sentence1'], example['sentence2'], truncation = True)

In [6]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)
data_collator = DataCollatorWithPadding(tokenizer= tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [7]:
tokenized_datasets = tokenized_datasets.remove_columns(['sentence1','sentence2','idx'])
tokenized_datasets = tokenized_datasets.rename_column('label','labels')
tokenized_datasets.set_format('torch')
tokenized_datasets['train'].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [8]:
#define data loaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets['train'], shuffle=True,batch_size = 8, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets['validation'], batch_size = 8, collate_fn = data_collator
)

In [12]:
for batch in train_dataloader:
  break
{k:v.shape for k,v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 77]),
 'token_type_ids': torch.Size([8, 77]),
 'attention_mask': torch.Size([8, 77])}

In [13]:
#create model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(1.1079, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [16]:
#define optimizer
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
#set hyperparameters
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)
print(num_training_steps)

1377


In [19]:
#get device
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
device

device(type='cuda')

In [20]:
#train loop
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k:v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [21]:
#eval loop
import evaluate
metric = evaluate.load('glue','mrpc')
model.eval()
for batch in eval_dataloader:
  batch = {k:v.to(device) for k,v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)
  logits = outputs.logits
  predictions = torch.argmax(logits, dim = -1)
  metric.add_batch(predictions = predictions, references = batch['labels'])
metric.compute()

{'accuracy': 0.8529411764705882, 'f1': 0.8969072164948454}

In [22]:
#Testing models
from transformers import pipeline
camembert_fill_mask = pipeline('fill-mask', model = 'camembert-base')
results = camembert_fill_mask('Le camembert est <mask> :)')

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Device set to use cuda:0


In [23]:
results

[{'score': 0.5238977074623108,
  'token': 7200,
  'token_str': 'délicieux',
  'sequence': 'Le camembert est délicieux :)'},
 {'score': 0.09637722373008728,
  'token': 2183,
  'token_str': 'excellent',
  'sequence': 'Le camembert est excellent :)'},
 {'score': 0.0363243892788887,
  'token': 26202,
  'token_str': 'succulent',
  'sequence': 'Le camembert est succulent :)'},
 {'score': 0.029079915955662727,
  'token': 528,
  'token_str': 'meilleur',
  'sequence': 'Le camembert est meilleur :)'},
 {'score': 0.027218375355005264,
  'token': 1654,
  'token_str': 'parfait',
  'sequence': 'Le camembert est parfait :)'}]

In [25]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained('camembert-base')
model = AutoModelForMaskedLM.from_pretrained('camembert-base')


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [58]:
inputs = tokenizer('Le camembert est <mask> :)', return_tensors='pt')
#input_tensor = torch.tensor(inputs['input_ids'])
#attention_tensor = torch.tensor(inputs['attention_mask'])


In [41]:
input_tensor = input_tensor.unsqueeze(0)

In [45]:
attention_tensor = attention_tensor.unsqueeze(0)

In [59]:
# Forward pass to get logits
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits

In [60]:
mask_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]

In [61]:
predicted_token_id = logits[0, mask_index].argmax(axis=-1)
predicted_word = tokenizer.decode(predicted_token_id)

In [63]:
sentence = 'Le camembert est <mask> :)'
filled_sentence = sentence.replace("<mask>", predicted_word)
print("Predicted Sentence:", filled_sentence)

Predicted Sentence: Le camembert est délicieux :)


In [67]:
top_k = 5  # Get the top 5 predictions

# Find the index of the <mask> token
mask_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]

# Get the top-k token predictions
top_k_probabilities, top_k_token_ids = torch.topk(logits[0, mask_index], top_k, dim=-1)

# Decode the predicted words
top_k_words = [tokenizer.decode(token_id.item()).strip() for token_id in top_k_token_ids[0]]

# Display results
print("Top predictions for <mask>:")
for i, word in enumerate(top_k_words, 1):
    print(f"{i}. {word}")


Top predictions for <mask>:
1. délicieux
2. excellent
3. succulent
4. meilleur
5. parfait
